In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'geum_auto_13'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, True, 20)

In [12]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

#ddday = 31
#mmmonth = 12

#length = 1

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    #df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
#                               colum_idx[idx], interpolation=interpolation_option[idx],
#                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2013.xlsx', '현도_2014.xlsx', '현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2013.xlsx', '대청호_2014.xlsx', '대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2013.xlsx', '장계_2014.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2013.xlsx
data/geum/자동/현도_2014.xlsx
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2013-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/대청호_2013.xlsx
data/geum/자동/대청호_2014.xlsx
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_201

1/1 [==============================] - 0s 20ms/step - gen_loss: 11.1371 - disc_loss: 0.3745 - rmse: 0.3706 - val_loss: 0.3823
Epoch 49/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0528 - disc_loss: 0.3482 - rmse: 0.3321 - val_loss: 0.3934
Epoch 50/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 12.1380 - disc_loss: 0.3483 - rmse: 0.4271 - val_loss: 0.3587
Epoch 51/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.8700 - disc_loss: 0.3360 - rmse: 0.3307 - val_loss: 0.3384
Epoch 52/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6132 - disc_loss: 0.3468 - rmse: 0.3399 - val_loss: 0.3336
Epoch 53/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5360 - disc_loss: 0.3517 - rmse: 0.2925 - val_loss: 0.3762
Epoch 54/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.2440 - disc_loss: 0.3419 - rmse: 0.3628 - val_loss: 0.3342
Epoch 55/2000
1/1 [====================

Epoch 107/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.4449 - disc_loss: 0.3089 - rmse: 0.2845 - val_loss: 0.2563
Epoch 108/2000
1/1 [==============================] - 0s 22ms/step - gen_loss: 4.8623 - disc_loss: 0.3003 - rmse: 0.2612 - val_loss: 0.2940
Epoch 109/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0981 - disc_loss: 0.3112 - rmse: 0.3052 - val_loss: 0.4065
Epoch 110/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0926 - disc_loss: 0.2757 - rmse: 0.2540 - val_loss: 0.2670
Epoch 111/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.6831 - disc_loss: 0.2974 - rmse: 0.3468 - val_loss: 0.2465
Epoch 112/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2616 - disc_loss: 0.2825 - rmse: 0.3202 - val_loss: 0.2696
Epoch 113/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 7.3601 - disc_loss: 0.2793 - rmse: 0.3187 - val_loss: 0.2378
Epoch 114/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 6.0881 - disc_loss: 0.2510 - rmse: 0.3111 - val_loss: 0.2252
Epoch 166/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7734 - disc_loss: 0.2695 - rmse: 0.2651 - val_loss: 0.2312
Epoch 167/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.6564 - disc_loss: 0.2830 - rmse: 0.5174 - val_loss: 0.2776
Epoch 168/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 6.6198 - disc_loss: 0.2577 - rmse: 0.3283 - val_loss: 0.2751
Epoch 169/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 16.4050 - disc_loss: 0.2616 - rmse: 0.5482 - val_loss: 0.2882
Epoch 170/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4146 - disc_loss: 0.2523 - rmse: 0.2910 - val_loss: 0.3177
Epoch 171/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 10.7501 - disc_loss: 0.2644 - rmse: 0.5413 - val_loss: 0.2642
Epoch 172/2000
1/1 [============

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7305 - disc_loss: 0.2367 - rmse: 0.2374 - val_loss: 0.2221
Epoch 224/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0451 - disc_loss: 0.2423 - rmse: 0.3292 - val_loss: 0.2803
Epoch 225/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.9626 - disc_loss: 0.2398 - rmse: 0.3100 - val_loss: 0.2202
Epoch 226/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7505 - disc_loss: 0.2495 - rmse: 0.2487 - val_loss: 0.2751
Epoch 227/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4926 - disc_loss: 0.2251 - rmse: 0.2325 - val_loss: 0.2845
Epoch 228/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3455 - disc_loss: 0.2441 - rmse: 0.2572 - val_loss: 0.3550
Epoch 229/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0175 - disc_loss: 0.2517 - rmse: 0.3099 - val_loss: 0.3275
Epoch 230/2000
1/1 [===============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3794 - disc_loss: 0.1745 - rmse: 0.1934 - val_loss: 0.1915
Epoch 1674/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4742 - disc_loss: 0.1634 - rmse: 0.1930 - val_loss: 0.1752
Epoch 1675/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.0696 - disc_loss: 0.1584 - rmse: 0.1661 - val_loss: 0.1823
Epoch 1676/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6809 - disc_loss: 0.1631 - rmse: 0.1910 - val_loss: 0.3847
Epoch 1677/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3181 - disc_loss: 0.1490 - rmse: 0.1884 - val_loss: 0.3422
Epoch 1678/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.6450 - disc_loss: 0.1453 - rmse: 0.3144 - val_loss: 0.2119
Epoch 1679/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.0584 - disc_loss: 0.1620 - rmse: 0.1527 - val_loss: 0.1681
Epoch 1680/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 17.1023 - disc_loss: 0.1569 - rmse: 0.2921 - val_loss: 0.2136
Epoch 1732/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.8173 - disc_loss: 0.1547 - rmse: 0.3798 - val_loss: 0.2447
Epoch 1733/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.1379 - disc_loss: 0.1652 - rmse: 0.2330 - val_loss: 0.3779
Epoch 1734/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5325 - disc_loss: 0.1622 - rmse: 0.3279 - val_loss: 0.2210
Epoch 1735/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.5318 - disc_loss: 0.1563 - rmse: 0.1885 - val_loss: 0.2238
Epoch 1736/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7468 - disc_loss: 0.1406 - rmse: 0.2004 - val_loss: 0.2069
Epoch 1737/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7229 - disc_loss: 0.1496 - rmse: 0.2056 - val_loss: 0.1959
Epoch 1738/2000
1/1 [======

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6569 - disc_loss: 0.1589 - rmse: 0.2226 - val_loss: 0.2185
Epoch 1790/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7974 - disc_loss: 0.1645 - rmse: 0.1874 - val_loss: 0.5101
Epoch 1791/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.5225 - disc_loss: 0.1588 - rmse: 0.2207 - val_loss: 0.2338
Epoch 1792/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3821 - disc_loss: 0.1485 - rmse: 0.2281 - val_loss: 0.2006
Epoch 1793/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.4346 - disc_loss: 0.1548 - rmse: 0.1678 - val_loss: 0.2879
Epoch 1794/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.2016 - disc_loss: 0.1647 - rmse: 0.2623 - val_loss: 0.2128
Epoch 1795/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3224 - disc_loss: 0.1473 - rmse: 0.1755 - val_loss: 0.1803
Epoch 1796/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1769 - disc_loss: 0.1504 - rmse: 0.2000 - val_loss: 0.4390
Epoch 1848/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.2503 - disc_loss: 0.1525 - rmse: 0.1819 - val_loss: 0.1805
Epoch 1849/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.8831 - disc_loss: 0.1521 - rmse: 0.2521 - val_loss: 0.2126
Epoch 1850/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.6080 - disc_loss: 0.1505 - rmse: 0.2430 - val_loss: 0.1821
Epoch 1851/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7938 - disc_loss: 0.1530 - rmse: 0.2210 - val_loss: 0.1941
Epoch 1852/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.1773 - disc_loss: 0.1543 - rmse: 0.1566 - val_loss: 0.3666
Epoch 1853/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6721 - disc_loss: 0.1532 - rmse: 0.2236 - val_loss: 0.2786
Epoch 1854/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3728 - disc_loss: 0.1498 - rmse: 0.2159 - val_loss: 0.1934
Epoch 1906/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 16.6434 - disc_loss: 0.1535 - rmse: 0.3271 - val_loss: 0.3274
Epoch 1907/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3918 - disc_loss: 0.1609 - rmse: 0.2322 - val_loss: 0.2873
Epoch 1908/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.7299 - disc_loss: 0.1549 - rmse: 0.2403 - val_loss: 0.2063
Epoch 1909/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.8808 - disc_loss: 0.1458 - rmse: 0.2381 - val_loss: 0.2946
Epoch 1910/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.5776 - disc_loss: 0.1568 - rmse: 0.2008 - val_loss: 0.2377
Epoch 1911/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.6300 - disc_loss: 0.1589 - rmse: 0.2404 - val_loss: 0.2403
Epoch 1912/2000
1/1 [=======

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [75]:
print(real_df_all.shape)
rnn_target_column = 'toc'
print(rnn_target_column)

#df_length = real_df_all.shape[0] - 3000
df_length = real_df_all.shape[0]

fake_df = real_df_all.iloc[:df_length,:]

(70128, 36)
toc


In [76]:
#train_df, val_df, test_df, test_df2 = dataset_slice(fake_df, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, test=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, test=1, val=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (49089, 36) val_df.shape :  (14026, 36) test_df.shape: (7013, 36)


In [77]:
#fake_df.shape[0]/8760

In [78]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [79]:
plt.figure()
plt.plot(val_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [80]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [81]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  4
out_num_features :  1


In [82]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [83]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    #if not os.path.exists('save/' + watershed):
    #    os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/geum_auto_13/toc/


In [84]:
#"save/" + watershed + "models/" + file_parameters['watershed'] + '/' + pa[indices[target_col_idx]]
#model_path

In [85]:
rnn_epochs

20

## 모델 학습

In [86]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=50,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 0.4337 - mean_absolute_error: 0.5491 - nse: -0.3898
Epoch 00001: val_nse improved from -inf to -0.33428, saving model to save/best_model.h5
10/10 [==============================] - 10s 982ms/step - loss: 0.4337 - mean_absolute_error: 0.5491 - nse: -0.3898 - val_loss: 0.3607 - val_mean_absolute_error: 0.4912 - val_nse: -0.3343
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 0.1347 - mean_absolute_error: 0.2954 - nse: 0.5609
Epoch 00002: val_nse improved from -0.33428 to 0.35628, saving model to save/best_model.h5
10/10 [==============================] - 10s 956ms/step - loss: 0.1347 - mean_absolute_error: 0.2954 - nse: 0.5609 - val_loss: 0.1735 - val_mean_absolute_error: 0.3444 - val_nse: 0.3563
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 0.0966 - mean_absolute_error: 0.2391 - nse: 0.6813
Epoch 00003: val_nse improved from 0.35628 to 0.65937, saving model to save/best_model.h5

Epoch 22/50
10/10 [==============================] - ETA: 0s - loss: 0.0454 - mean_absolute_error: 0.1528 - nse: 0.8580
Epoch 00022: val_nse did not improve from 0.82005
10/10 [==============================] - 10s 967ms/step - loss: 0.0454 - mean_absolute_error: 0.1528 - nse: 0.8580 - val_loss: 0.0784 - val_mean_absolute_error: 0.2148 - val_nse: 0.7113
Epoch 23/50
10/10 [==============================] - ETA: 0s - loss: 0.0467 - mean_absolute_error: 0.1550 - nse: 0.8543
Epoch 00023: val_nse did not improve from 0.82005
10/10 [==============================] - 10s 962ms/step - loss: 0.0467 - mean_absolute_error: 0.1550 - nse: 0.8543 - val_loss: 0.0914 - val_mean_absolute_error: 0.2237 - val_nse: 0.6636
Epoch 24/50
10/10 [==============================] - ETA: 0s - loss: 0.0439 - mean_absolute_error: 0.1503 - nse: 0.8589
Epoch 00024: val_nse did not improve from 0.82005
10/10 [==============================] - 10s 952ms/step - loss: 0.0439 - mean_absolute_error: 0.1503 - nse: 0.8589 - v

10/10 [==============================] - ETA: 0s - loss: 0.0415 - mean_absolute_error: 0.1476 - nse: 0.8713
Epoch 00045: val_nse did not improve from 0.82005
10/10 [==============================] - 10s 964ms/step - loss: 0.0415 - mean_absolute_error: 0.1476 - nse: 0.8713 - val_loss: 0.1292 - val_mean_absolute_error: 0.2650 - val_nse: 0.5232
Epoch 46/50
10/10 [==============================] - ETA: 0s - loss: 0.0391 - mean_absolute_error: 0.1444 - nse: 0.8807
Epoch 00046: val_nse did not improve from 0.82005
10/10 [==============================] - 10s 961ms/step - loss: 0.0391 - mean_absolute_error: 0.1444 - nse: 0.8807 - val_loss: 0.1400 - val_mean_absolute_error: 0.2719 - val_nse: 0.4856
Epoch 47/50
10/10 [==============================] - ETA: 0s - loss: 0.0422 - mean_absolute_error: 0.1478 - nse: 0.8686
Epoch 00047: val_nse did not improve from 0.82005
10/10 [==============================] - 10s 962ms/step - loss: 0.0422 - mean_absolute_error: 0.1478 - nse: 0.8686 - val_loss: 0.1

## core / window.py / 

In [87]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  2.286552
predict_day :  5
------------------------
0.09970784 0.01703396 0.13051422
nse3 :  [0.7707672]
pbias3 :  [2.4875143]
save/geum/models/geum_auto_13/toc/
year : 2013 ~ 2020
run :  range(0, 1)
target :  toc
length :  70128
train=7, test=1, val=2
---------------
[0.7707672]
[2.4875143]


In [72]:
#train_df.shape, val_df.shape, test_df.shape

In [73]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvbnfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
from core.models import GRUModel, compile

In [ ]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [ ]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    #model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    model=model112,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print("length : ",df_length)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

In [ ]:
##.evaluate(multi_window.train)
#model112.evaluate(multi_window.val.repeat(-1), steps=100)

In [ ]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [ ]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()
